In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
from prophet import Prophet
from minio import Minio
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional
from keras.callbacks import EarlyStopping
# evaluate an xgboost regression model on the housing dataset
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
import seaborn as sns
import json
from openai import OpenAI


# MINIO_ENDPOINT = os.environ.get("MINIO_ENDPOINT")
# MINIO_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY")
# MINIO_SECRET_KEY = os.environ.get("MINIO_SECRET_KEY")

# print(MINIO_ENDPOINT)

# minio_client = Minio(
#     endpoint=MINIO_ENDPOINT,
#     access_key=MINIO_ACCESS_KEY,
#     secret_key=MINIO_SECRET_KEY,
#     secure=False
# )

2024-08-29 13:54:38.173919: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("customer_shopping_data.csv")

In [15]:
user_columns = df.columns
user_columns = list(user_columns) if type(user_columns) is not list else user_columns
print(user_columns)

['invoice_no', 'customer_id', 'gender', 'age', 'category', 'quantity', 'price', 'payment_method', 'invoice_date', 'shopping_mall']


In [3]:
TARGET_LIST = ['Transaction ID', 'Date', 'Time', 'Customer ID', 'Product ID', 'Product Name', 'Category', 'Quantity', 'Unit Price', 'Total Price', 'Payment Method', 'Store Location', 'Discount', 'Salesperson ID', 'Profit Margin', 'Customer Gender', 'Customer Age']

prompt = f"""Map the following columns with the list that has the same meaning, if do not found keep the original name of input column:

Columns: {list(df.columns)}

JSON list: {TARGET_LIST}

Just repsonse JSON-like format"""
print(prompt)


Map the following columns with the list that has the same meaning, if do not found keep the original name of input column:

Columns: ['invoice_no', 'customer_id', 'gender', 'age', 'category', 'quantity', 'price', 'payment_method', 'invoice_date', 'shopping_mall']

JSON list: ['Transaction ID', 'Date', 'Time', 'Customer ID', 'Product ID', 'Product Name', 'Category', 'Quantity', 'Unit Price', 'Total Price', 'Payment Method', 'Store Location', 'Discount', 'Salesperson ID', 'Profit Margin', 'Customer Gender', 'Customer Age']

Just repsonse JSON-like format


In [4]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
print(api_key)


sk-jPvLv5UrgeUJVThuh9whAGV2EwjO_Jqdtg0LdyhNEVT3BlbkFJd1cpoqQuHHsZGG7fkysqXstLt2vBKgCTln9L8YaggA


In [5]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
print(api_key)

client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
  ]
)

sk-jPvLv5UrgeUJVThuh9whAGV2EwjO_Jqdtg0LdyhNEVT3BlbkFJd1cpoqQuHHsZGG7fkysqXstLt2vBKgCTln9L8YaggA


In [7]:
message = response.choices[0].message.content

In [17]:

json.loads(message)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
df.head()

invoice_no customer_id  gender  age  category  quantity    price  \
0    I138884     C241288  Female   28  Clothing         5  1500.40   
1    I317333     C111565    Male   21     Shoes         3  1800.51   
2    I127801     C266599    Male   20  Clothing         1   300.08   
3    I173702     C988172  Female   66     Shoes         5  3000.85   
4    I337046     C189076  Female   53     Books         4    60.60   

  payment_method invoice_date   shopping_mall  
0    Credit Card     5/8/2022          Kanyon  
1     Debit Card   12/12/2021  Forum Istanbul  
2           Cash    9/11/2021       Metrocity  
3    Credit Card   16/05/2021    Metropol AVM  
4           Cash   24/10/2021          Kanyon

In [6]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_in -= 1
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # Input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]


    cols.append(df.shift(0))
    names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
    
    cols.append(df.shift(-n_out))
    names += [('var%d(t+%d)' % (j+1, n_out)) for j in range(n_vars)]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [7]:
def try_parse_datetime(data, formats):
    for fmt in formats:
        try:
            return pd.to_datetime(data, format=fmt)
        except ValueError:
            continue
    # if fail return an empty series
    return pd.Series()


def data_preprocessing(df: pd.DataFrame):
    if "Total Price" not in df.columns:
        if "Unit Price" in df.columns and "Quantity" in df.columns:
            try:
                df['Total Price'] = df['Quantity'] * df['Unit Price']
            except(Exception):
                print(Exception)
    if "Date" in df.columns:
        # Try parse multiple datetime until success, if
        formats = ['%d/%m/%Y']
        datetime = try_parse_datetime(df['Date'], formats=formats)
      
        if not datetime.empty:
            df['Date'] = datetime
        df.sort_values(by='Date', inplace=True)
    return df



def mapping(user_columns: list[str], user_id: str):
    """
    Return mapping user columns and system column and store as JSON file.

            Parameters:
                    user_columns (list(str)): User columns
                    user_id (str): User ID
            Returns:
                    map (dict)
    """

    return {
            'invoice_no': 'Transaction ID',
            'customer_id': 'Customer ID',
            'gender': "Customer Gender",
            'age': "Customer Age",
            'category': 'Category',
            'quantity': 'Quantity',
            'price': 'Unit Price',
            'payment_method': 'Payment Method',
            'invoice_date': 'Date',
            'shopping_mall': 'Store Location'
        }


In [8]:
mapping = mapping([], [])

In [9]:
df = pd.read_csv("customer_shopping_data.csv")

In [10]:
df = df.rename(mapper=mapping, axis=1)

In [11]:
data_preprocessing(df)

Transaction ID Customer ID Customer Gender  Customer Age  \
36199        I192911     C642829          Female            66   
78539        I313757     C438201          Female            43   
27812        I836951     C382440            Male            54   
27856        I159235     C312481            Male            58   
5323         I115941     C105769            Male            34   
...              ...         ...             ...           ...   
20863        I218048     C389917          Female            66   
20879        I268432     C175797          Female            37   
61644        I200744     C224026          Female            59   
27871        I124836     C265661          Female            46   
60463        I154677     C188068          Female            49   

              Category  Quantity  Unit Price Payment Method       Date  \
36199  Food & Beverage         5       26.15    Credit Card 2021-01-01   
78539         Clothing         4     1200.32    Credit Card 2021-01-01   
27812         Clothing         3      900.24           Cash 2021-01-01   
27856  Food & Beverage         4       20.92    Credit Card 2021-01-01   
5323          Clothing         1      300.08           Cash 2021-01-01   
...                ...       ...         ...            ...        ...   
20863         Clothing         5     1500.40           Cash 2023-03-08   
20879  Food & Beverage         3       15.69           Cash 2023-03-08   
61644        Cosmetics         5      203.30    Credit Card 2023-03-08   
27871            Shoes         3     1800.51    Credit Card 2023-03-08   
60463        Cosmetics         1       40.66           Cash 2023-03-08   

         Store Location  Total Price  
36199         Metrocity       130.75  
78539  Mall of Istanbul      4801.28  
27812         Metrocity      2700.72  
27856  Mall of Istanbul        83.68  
5323       Istinye Park       300.08  
...                 ...          ...  
20863  Mall of Istanbul      7502.00  
20879            Kanyon        47.07  
61644            Kanyon      1016.50  
27871         Metrocity      5401.53  
60463            Kanyon        40.66  

[99457 rows x 11 columns]

In [12]:
df.head()

Transaction ID Customer ID Customer Gender  Customer Age  \
36199        I192911     C642829          Female            66   
78539        I313757     C438201          Female            43   
27812        I836951     C382440            Male            54   
27856        I159235     C312481            Male            58   
5323         I115941     C105769            Male            34   

              Category  Quantity  Unit Price Payment Method       Date  \
36199  Food & Beverage         5       26.15    Credit Card 2021-01-01   
78539         Clothing         4     1200.32    Credit Card 2021-01-01   
27812         Clothing         3      900.24           Cash 2021-01-01   
27856  Food & Beverage         4       20.92    Credit Card 2021-01-01   
5323          Clothing         1      300.08           Cash 2021-01-01   

         Store Location  Total Price  
36199         Metrocity       130.75  
78539  Mall of Istanbul      4801.28  
27812         Metrocity      2700.72  
27856  Mall of Istanbul        83.68  
5323       Istinye Park       300.08

In [13]:
if not None:
    print("Hello")

Hello


In [14]:
gb_df = df.groupby(by="Date")['Total Price'].sum()

In [15]:
gb_df

Date
2021-01-01    271192.66
2021-01-02    303659.14
2021-01-03    375794.99
2021-01-04    284512.26
2021-01-05    308395.08
                ...    
2023-03-04    402423.09
2023-03-05    305423.09
2023-03-06    442091.99
2023-03-07    213393.21
2023-03-08    322808.90
Name: Total Price, Length: 797, dtype: float64

In [16]:
df.head()

Transaction ID Customer ID Customer Gender  Customer Age  \
36199        I192911     C642829          Female            66   
78539        I313757     C438201          Female            43   
27812        I836951     C382440            Male            54   
27856        I159235     C312481            Male            58   
5323         I115941     C105769            Male            34   

              Category  Quantity  Unit Price Payment Method       Date  \
36199  Food & Beverage         5       26.15    Credit Card 2021-01-01   
78539         Clothing         4     1200.32    Credit Card 2021-01-01   
27812         Clothing         3      900.24           Cash 2021-01-01   
27856  Food & Beverage         4       20.92    Credit Card 2021-01-01   
5323          Clothing         1      300.08           Cash 2021-01-01   

         Store Location  Total Price  
36199         Metrocity       130.75  
78539  Mall of Istanbul      4801.28  
27812         Metrocity      2700.72  
27856  Mall of Istanbul        83.68  
5323       Istinye Park       300.08

In [17]:
gb_df.describe()

count       797.000000
mean     315565.613864
std       54995.142183
min      171339.480000
25%      276255.240000
50%      312005.520000
75%      350833.120000
max      534906.860000
Name: Total Price, dtype: float64

In [18]:


def train_with_prophet(data):
    # Prepare the data for Prophet
    prophet_data = data.groupby("Date")['Quantity'].sum()
    prophet_train = prophet_data.reset_index()
    prophet_train = prophet_train.rename(columns={"Date": "ds", "Quantity": "y"})
    prophet_train["ds"] = pd.to_datetime(prophet_train["ds"])

    # Split the data into training and testing sets
    index = int(0.8 * len(prophet_train))
    train = prophet_train.iloc[:index]
    test = prophet_train.iloc[index:]
    print(test[:5])
    # Train the Prophet model
    m = Prophet()
    m.fit(train)

    # Make predictions
    forecast = m.predict(test)

    # Calculate mean squared error
    mse = mean_squared_error(test['y'], forecast['yhat'])
    print("Mean squared error:", mse)

    # Plotting with Matplotlib
    plt.figure(figsize=(12, 6))
    plt.plot(test['ds'], test['y'], label='Real', color='blue')
    plt.plot(forecast['ds'], forecast['yhat'], label='Predicted', color='orange')

    # Formatting the plot
    plt.title('Real vs Predicted Values')

    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Show the plot
    plt.show()

# Example usage
# Assuming `data` is a DataFrame with a 'Date' and 'Quantity' column
# train_with_prophet(data)


In [19]:
train_with_prophet(df)

14:28:14 - cmdstanpy - INFO - Chain [1] start processing
14:28:14 - cmdstanpy - INFO - Chain [1] done processing


            ds    y
637 2022-09-30  370
638 2022-10-01  392
639 2022-10-02  435
640 2022-10-03  415
641 2022-10-04  327
Mean squared error: 1257.9419410219039


In [20]:
prophet_data = df.groupby("Date")['Quantity'].sum()
prophet_train = prophet_data.reset_index()
prophet_train = prophet_train.rename(columns={"Date": "ds", "Quantity": "y"})
prophet_train["ds"] = pd.to_datetime(prophet_train["ds"])

In [21]:
index = int(0.8 * len(prophet_train))
train = prophet_train.iloc[:index]
test = prophet_train.iloc[index:]

# Train the Prophet model
m = Prophet()
m.fit(train)

# Make predictions
forecast = m.predict(test)

14:28:15 - cmdstanpy - INFO - Chain [1] start processing


14:28:15 - cmdstanpy - INFO - Chain [1] done processing


In [22]:
forecast[['ds', 'yhat']]

ds        yhat
0   2022-09-30  381.474207
1   2022-10-01  373.879261
2   2022-10-02  369.779076
3   2022-10-03  384.845110
4   2022-10-04  380.755926
..         ...         ...
155 2023-03-04  375.711343
156 2023-03-05  371.611157
157 2023-03-06  386.677191
158 2023-03-07  382.588007
159 2023-03-08  379.696721

[160 rows x 2 columns]

In [23]:
mean_squared_error(test['y'], forecast['yhat'])

1257.9419410219039

In [24]:
df.head()

Transaction ID Customer ID Customer Gender  Customer Age  \
36199        I192911     C642829          Female            66   
78539        I313757     C438201          Female            43   
27812        I836951     C382440            Male            54   
27856        I159235     C312481            Male            58   
5323         I115941     C105769            Male            34   

              Category  Quantity  Unit Price Payment Method       Date  \
36199  Food & Beverage         5       26.15    Credit Card 2021-01-01   
78539         Clothing         4     1200.32    Credit Card 2021-01-01   
27812         Clothing         3      900.24           Cash 2021-01-01   
27856  Food & Beverage         4       20.92    Credit Card 2021-01-01   
5323          Clothing         1      300.08           Cash 2021-01-01   

         Store Location  Total Price  
36199         Metrocity       130.75  
78539  Mall of Istanbul      4801.28  
27812         Metrocity      2700.72  
27856  Mall of Istanbul        83.68  
5323       Istinye Park       300.08

In [25]:
data = df.groupby("Date")['Quantity'].sum().reset_index().set_index("Date")

In [26]:
type(data)

pandas.core.frame.DataFrame

In [27]:
data.head()

Quantity
Date                
2021-01-01       333
2021-01-02       387
2021-01-03       405
2021-01-04       379
2021-01-05       374

In [28]:
# Convert the time series data to a supervised learning format
n_in = 30  # Number of lag observations as input
n_out = 30  # Number of observations as output

supervised_data = series_to_supervised(data[["Quantity"]], n_in, n_out)

# Separate the input features (X) and the output target (y)


In [29]:
data.head()

Quantity
Date                
2021-01-01       333
2021-01-02       387
2021-01-03       405
2021-01-04       379
2021-01-05       374

In [30]:
# series_to_supervised(data[["Quantity"]], n_in, n_out, False).tail(30)

In [31]:
supervised_data

var1(t-29)  var1(t-28)  var1(t-27)  var1(t-26)  var1(t-25)  \
Date                                                                     
2021-01-30       333.0       387.0       405.0       379.0       374.0   
2021-01-31       387.0       405.0       379.0       374.0       459.0   
2021-02-01       405.0       379.0       374.0       459.0       402.0   
2021-02-02       379.0       374.0       459.0       402.0       337.0   
2021-02-03       374.0       459.0       402.0       337.0       322.0   
...                ...         ...         ...         ...         ...   
2023-02-02       361.0       441.0       397.0       436.0       363.0   
2023-02-03       441.0       397.0       436.0       363.0       391.0   
2023-02-04       397.0       436.0       363.0       391.0       361.0   
2023-02-05       436.0       363.0       391.0       361.0       328.0   
2023-02-06       363.0       391.0       361.0       328.0       373.0   

            var1(t-24)  var1(t-23)  var1(t-22)  var1(t-21)  var1(t-20)  ...  \
Date                                                                    ...   
2021-01-30       459.0       402.0       337.0       322.0       351.0  ...   
2021-01-31       402.0       337.0       322.0       351.0       344.0  ...   
2021-02-01       337.0       322.0       351.0       344.0       378.0  ...   
2021-02-02       322.0       351.0       344.0       378.0       369.0  ...   
2021-02-03       351.0       344.0       378.0       369.0       378.0  ...   
...                ...         ...         ...         ...         ...  ...   
2023-02-02       391.0       361.0       328.0       373.0       392.0  ...   
2023-02-03       361.0       328.0       373.0       392.0       343.0  ...   
2023-02-04       328.0       373.0       392.0       343.0       354.0  ...   
2023-02-05       373.0       392.0       343.0       354.0       375.0  ...   
2023-02-06       392.0       343.0       354.0       375.0       369.0  ...   

            var1(t-8)  var1(t-7)  var1(t-6)  var1(t-5)  var1(t-4)  var1(t-3)  \
Date                                                                           
2021-01-30      408.0      373.0      347.0      379.0      348.0      363.0   
2021-01-31      373.0      347.0      379.0      348.0      363.0      389.0   
2021-02-01      347.0      379.0      348.0      363.0      389.0      330.0   
2021-02-02      379.0      348.0      363.0      389.0      330.0      363.0   
2021-02-03      348.0      363.0      389.0      330.0      363.0      321.0   
...               ...        ...        ...        ...        ...        ...   
2023-02-02      379.0      373.0      392.0      362.0      378.0      391.0   
2023-02-03      373.0      392.0      362.0      378.0      391.0      375.0   
2023-02-04      392.0      362.0      378.0      391.0      375.0      388.0   
2023-02-05      362.0      378.0      391.0      375.0      388.0      437.0   
2023-02-06      378.0      391.0      375.0      388.0      437.0      344.0   

            var1(t-2)  var1(t-1)  var1(t)  var1(t+30)  
Date                                                   
2021-01-30      389.0      330.0      363       303.0  
2021-01-31      330.0      363.0      321       342.0  
2021-02-01      363.0      321.0      330       348.0  
2021-02-02      321.0      330.0      329       402.0  
2021-02-03      330.0      329.0      353       372.0  
...               ...        ...      ...         ...  
2023-02-02      375.0      388.0      437       435.0  
2023-02-03      388.0      437.0      344       354.0  
2023-02-04      437.0      344.0      404       401.0  
2023-02-05      344.0      404.0      362       306.0  
2023-02-06      404.0      362.0      383       320.0  

[738 rows x 31 columns]

In [32]:
from xgboost import XGBRegressor

In [33]:
data[["Quantity"]].head()

Quantity
Date                
2021-01-01       333
2021-01-02       387
2021-01-03       405
2021-01-04       379
2021-01-05       374

In [34]:
supervised_data.head()

var1(t-29)  var1(t-28)  var1(t-27)  var1(t-26)  var1(t-25)  \
Date                                                                     
2021-01-30       333.0       387.0       405.0       379.0       374.0   
2021-01-31       387.0       405.0       379.0       374.0       459.0   
2021-02-01       405.0       379.0       374.0       459.0       402.0   
2021-02-02       379.0       374.0       459.0       402.0       337.0   
2021-02-03       374.0       459.0       402.0       337.0       322.0   

            var1(t-24)  var1(t-23)  var1(t-22)  var1(t-21)  var1(t-20)  ...  \
Date                                                                    ...   
2021-01-30       459.0       402.0       337.0       322.0       351.0  ...   
2021-01-31       402.0       337.0       322.0       351.0       344.0  ...   
2021-02-01       337.0       322.0       351.0       344.0       378.0  ...   
2021-02-02       322.0       351.0       344.0       378.0       369.0  ...   
2021-02-03       351.0       344.0       378.0       369.0       378.0  ...   

            var1(t-8)  var1(t-7)  var1(t-6)  var1(t-5)  var1(t-4)  var1(t-3)  \
Date                                                                           
2021-01-30      408.0      373.0      347.0      379.0      348.0      363.0   
2021-01-31      373.0      347.0      379.0      348.0      363.0      389.0   
2021-02-01      347.0      379.0      348.0      363.0      389.0      330.0   
2021-02-02      379.0      348.0      363.0      389.0      330.0      363.0   
2021-02-03      348.0      363.0      389.0      330.0      363.0      321.0   

            var1(t-2)  var1(t-1)  var1(t)  var1(t+30)  
Date                                                   
2021-01-30      389.0      330.0      363       303.0  
2021-01-31      330.0      363.0      321       342.0  
2021-02-01      363.0      321.0      330       348.0  
2021-02-02      321.0      330.0      329       402.0  
2021-02-03      330.0      329.0      353       372.0  

[5 rows x 31 columns]

In [35]:
X = supervised_data.iloc[:, :-1]
y = supervised_data.iloc[:, -1]

In [36]:
X.shape

(738, 30)

In [37]:
y

Date
2021-01-30    303.0
2021-01-31    342.0
2021-02-01    348.0
2021-02-02    402.0
2021-02-03    372.0
              ...  
2023-02-02    435.0
2023-02-03    354.0
2023-02-04    401.0
2023-02-05    306.0
2023-02-06    320.0
Name: var1(t+30), Length: 738, dtype: float64

In [38]:
# X = np.array(X.values)
# y = np.array(y.values)

In [39]:
index = int(0.8 * len(X))
X_train = X[:index, :]
y_train = y[:index]
X_test = X[index:, :]
y_test = y[index:]

InvalidIndexError: (slice(None, 590, None), slice(None, None, None))

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
model = Sequential([
    LSTM(32, return_sequences= True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(32, return_sequences= False),
    Dense(32, activation="relu"),
    Dense(1)
])

model.compile(optimizer= 'adam', loss= 'mse' , metrics= "mean_absolute_error")

2024-08-16 14:08:56.140241: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-16 14:08:56.141264: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-16 14:08:56.142447: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 1, 32)             8064      
                                                                 
 lstm_7 (LSTM)               (None, 32)                8320      
                                                                 
 dense_7 (Dense)             (None, 32)                1056      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 17,473
Trainable params: 17,473
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Fitting the LSTM to the Training set
callbacks = [EarlyStopping(monitor= 'loss', patience= 10 , restore_best_weights= True)]
history = model.fit(X_train, y_train, epochs= 100, batch_size= 32)

Epoch 1/100


2024-08-16 14:08:57.878259: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-16 14:08:57.879934: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-16 14:08:57.881788: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

19/19 [==============================] - 2s 11ms/step - loss: 141771.4375 - mean_absolute_error: 374.8292
Epoch 2/100
19/19 [==============================] - 0s 9ms/step - loss: 141432.8125 - mean_absolute_error: 374.3775
Epoch 3/100
19/19 [==============================] - 0s 5ms/step - loss: 140853.1875 - mean_absolute_error: 373.6009
Epoch 4/100
19/19 [==============================] - 0s 5ms/step - loss: 139813.7188 - mean_absolute_error: 372.2073
Epoch 5/100
19/19 [==============================] - 0s 5ms/step - loss: 138098.5312 - mean_absolute_error: 369.8961
Epoch 6/100
19/19 [==============================] - 0s 5ms/step - loss: 135573.8906 - mean_absolute_error: 366.4599
Epoch 7/100
19/19 [==============================] - 0s 5ms/step - loss: 132224.2188 - mean_absolute_error: 361.8684
Epoch 8/100
19/19 [==============================] - 0s 7ms/step - loss: 128167.7578 - mean_absolute_error: 356.2006
Epoch 9/100
19/19 [==============================] - 0s 8ms/step - loss: 12

In [ ]:
history.history['loss'][-10:]

[1274.8106689453125,
 1274.637939453125,
 1275.241943359375,
 1274.90625,
 1274.85693359375,
 1274.8934326171875,
 1274.7391357421875,
 1275.087890625,
 1274.8154296875,
 1274.67041015625]

In [ ]:
plt.plot(history.history["loss"][-50:])
plt.plot(history.history["mean_absolute_error"][-50:])
plt.legend(['Mean Squared Error','Mean Absolute Error'])
plt.title("Losses")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
predictions = model.predict(X_test)

5/5 [==============================] - 0s 2ms/step


In [ ]:
predictions

array([[374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],
       [374.92624],


In [ ]:
X_test[]

array([[[413., 376., 358., ..., 353., 374., 369.]],

       [[376., 358., 380., ..., 374., 369., 381.]],

       [[358., 380., 370., ..., 369., 381., 350.]],

       ...,

       [[397., 436., 363., ..., 437., 344., 404.]],

       [[436., 363., 391., ..., 344., 404., 362.]],

       [[363., 391., 361., ..., 404., 362., 383.]]])

In [ ]:
index = int(0.8 * len(X))
X_train = X[:index, :]
y_train = y[:index]
X_test = X[index:, :]
y_test = y[index:]

model = XGBRegressor()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )



Mean MAE: 31.472 (3.571)


In [ ]:
model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.1, eval_metric=None,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
y_hat = model.predict(X_test)

In [ ]:
print("Mean squared error:", mean_squared_error(y_hat, y_test))
print("Mean absolute error:", mean_absolute_error(y_hat, y_test))

Mean squared error: 1411.017891466435
Mean absolute error: 30.06064811912743


array([366.9508 , 347.2809 , 378.21683, 381.69275, 391.1118 , 366.71136,
       378.00543, 352.53873, 384.45563, 381.30472, 396.07828, 374.81323,
       384.67572, 360.64505, 398.93558, 376.6627 , 374.71338, 356.6004 ,
       372.54355, 341.01324, 355.1147 , 385.4757 , 381.6995 , 344.31808,
       377.54865, 377.6639 , 389.562  , 373.14157, 402.23013, 363.1917 ,
       374.49875, 376.5403 , 384.3461 , 360.82947, 360.31982, 376.92743,
       396.6947 , 392.31332, 350.6054 , 372.22723, 383.96097, 375.2373 ,
       390.50528, 381.71732, 350.3593 , 364.17294, 392.3082 , 362.96814,
       372.3977 , 354.17538, 388.976  , 353.91104, 357.62405, 359.0623 ,
       383.11447, 364.7931 , 364.5962 , 387.36993, 346.1965 , 362.09723,
       366.16272, 393.72153, 377.11578, 352.81113, 372.8995 , 350.491  ,
       390.39386, 388.52167, 344.07492, 385.83054, 367.84494, 369.97772,
       373.0625 , 384.0018 , 389.12167, 361.74655, 374.84302, 373.93558,
       376.87164, 370.16357, 394.4607 , 379.04535, 

In [ ]:
def plot_evaluation(ds, y_truth, y_hat):
    plt.figure(figsize=(12, 6))
    plt.plot(ds, y_truth, label='Real', color='blue')
    plt.plot(ds, y_hat, label='Predicted', color='orange')

    # Formatting the plot
    plt.title('Real vs Predicted Values')

    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
plot_evaluation(data.index[index + 59:], y_test, y_hat)

In [ ]:
y_hat.shape

(148,)

In [ ]:
len(data.index)

797

In [ ]:
type(pd.DataFrame({"ds": [1,2,3,4]})) is list

False

In [ ]:
pd.DataFrame({"ds": [1,2,3,4]})

ds
0   1
1   2
2   3
3   4

In [ ]:
def train_with_prophet(data, test_size, target):
    # Prepare the data for Prophet
    prophet_data = data.groupby("Date")[target].sum()
    prophet_train = prophet_data.reset_index()
    prophet_train = prophet_train.rename(columns={"Date": "ds", target: "y"})
    prophet_train["ds"] = pd.to_datetime(prophet_train["ds"])

    # Split the data into training and testing sets
    index = int(test_size * len(prophet_train))
    train = prophet_train.iloc[:index]
    test = prophet_train.iloc[index:]
    # Train the Prophet model
    model = Prophet()
    model.fit(train)

    # Make predictions
    forecast = model.predict(test)

    


In [45]:
pd.api.types.is_datetime64_dtype(data.index)

True

Transaction ID Customer ID Customer Gender  Customer Age  \
36199        I192911     C642829          Female            66   
78539        I313757     C438201          Female            43   
27812        I836951     C382440            Male            54   
27856        I159235     C312481            Male            58   
5323         I115941     C105769            Male            34   
...              ...         ...             ...           ...   
54153        I122847     C290329          Female            53   
386          I289611     C222906          Female            45   
38270        I139558     C196970            Male            41   
20863        I218048     C389917          Female            66   
20879        I268432     C175797          Female            37   

              Category  Quantity  Unit Price Payment Method       Date  \
36199  Food & Beverage         5       26.15    Credit Card 2021-01-01   
78539         Clothing         4     1200.32    Credit Card 2021-01-01   
27812         Clothing         3      900.24           Cash 2021-01-01   
27856  Food & Beverage         4       20.92    Credit Card 2021-01-01   
5323          Clothing         1      300.08           Cash 2021-01-01   
...                ...       ...         ...            ...        ...   
54153  Food & Beverage         4       20.92    Credit Card 2023-03-08   
386           Clothing         3      900.24     Debit Card 2023-03-08   
38270  Food & Beverage         4       20.92    Credit Card 2023-03-08   
20863         Clothing         5     1500.40           Cash 2023-03-08   
20879  Food & Beverage         3       15.69           Cash 2023-03-08   

         Store Location  Total Price  
36199         Metrocity       130.75  
78539  Mall of Istanbul      4801.28  
27812         Metrocity      2700.72  
27856  Mall of Istanbul        83.68  
5323       Istinye Park       300.08  
...                 ...          ...  
54153            Kanyon        83.68  
386         Cevahir AVM      2700.72  
38270         Metrocity        83.68  
20863  Mall of Istanbul      7502.00  
20879            Kanyon        47.07  

[49263 rows x 11 columns]

In [67]:
filter_1 = """[
  {
    "column": "Category",
    "type": "include",
    "values": [
      "Clothing",
      "Food & Beverage"
    ]
  },
]"""
filter_list = json.loads(filter_1)

filter_df = df.copy()

for filter in filter_list:
    column = filter['column']
    type = filter['type']
    if type == "include":
        filter_df = filter_df[filter_df[column].isin(filter['values'])]
    elif type == "daterange":
        from_date = pd.to_datetime(filter['from_to'][0])
        to_date = pd.to_datetime(filter['from_to'][1])
        filter_df = filter_df[(filter_df[column] >= from_date) & (filter_df[column] <= to_date)]
    elif type == "range":
        from_value = pd.to_numeric(filter['from_to'][0])
        to_value = pd.to_numeric(filter['from_to'][1])
        filter_df = filter_df[(filter_df[column] >= from_value) & (filter_df[column] <= to_value)]
filter_df.head()

Transaction ID Customer ID Customer Gender  Customer Age  Category  \
53698        I333688     C294917          Female            53  Clothing   
80438        I122430     C900891          Female            48  Clothing   
76555        I437114     C263408          Female            62  Clothing   
16115        I154725     C544940            Male            63  Clothing   
51976        I246792     C405158          Female            34  Clothing   

       Quantity  Unit Price Payment Method       Date     Store Location  \
53698         1      300.08           Cash 2022-03-08  Emaar Square Mall   
80438         1      300.08           Cash 2022-03-08        Cevahir AVM   
76555         2      600.16           Cash 2022-03-08       Istinye Park   
16115         2      600.16           Cash 2022-03-08   Mall of Istanbul   
51976         2      600.16           Cash 2022-03-08   Mall of Istanbul   

       Total Price  
53698       300.08  
80438       300.08  
76555      1200.32  
16115      1200.32  
51976      1200.32

In [68]:
json.loads('[]')

[]